<a href="https://colab.research.google.com/github/InowaR/colab/blob/main/polinomial_matrix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np

# Исходные данные
X = np.array([
    [5, 2, 1, 0], [6, 3, 2, 1], [4, 1, 1, 0],
    [7, 4, 6, 3], [8, 5, 5, 4], [6, 3, 4, 2],
    [3, 8, 7, 9], [4, 9, 8, 8], [2, 7, 6, 7],
], dtype=np.float32) / 10.0

y = np.array([0, 0, 0, 1, 1, 1, 1, 0, 1], dtype=np.float32)

# Функция для создания матрицы дизайна (всех термов)
def create_design_matrix(X):
    n_samples = X.shape[0]
    X1, X2, X3, X4 = X[:, 0], X[:, 1], X[:, 2], X[:, 3]

    # Создаем матрицу дизайна с полиномиальными термами 2-й степени
    design_matrix = np.column_stack([
        np.ones(n_samples, dtype=np.float32),  # bias term
        X1, X2, X3, X4,                       # линейные термы
        X1**2, X2**2, X3**2, X4**2,           # квадратичные термы
        X1*X2, X1*X3, X1*X4,                  # взаимодействия x1
        X2*X3, X2*X4, X3*X4                   # взаимодействия x2, x3, x4
    ])
    return design_matrix

# Создаем матрицу дизайна один раз
design_matrix = create_design_matrix(X)

# Инициализация весов
weights = np.random.uniform(-0.1, 0.1, size=design_matrix.shape[1]).astype(np.float32)

# Параметры обучения
learning_rate = 0.1
n_iterations = 10000

# Обучение с матричными операциями
for _ in range(n_iterations):
    # Вычисляем предсказания для всех образцов сразу
    predictions = np.dot(design_matrix, weights)

    # Вычисляем ошибку для всех образцов
    errors = predictions - y

    # Обновляем веса (матричная операция)
    gradient = np.dot(design_matrix.T, errors) / len(y)
    weights -= learning_rate * gradient

print("Результаты:")
predictions = np.dot(design_matrix, weights)
for i in range(len(X)):
    print(f"{X[i] * 10} -> {predictions[i]:.4f} (Истинное: {y[i]})")

# Можно также вывести среднеквадратичную ошибку
mse = np.mean((predictions - y)**2)
print(f"\nСреднеквадратичная ошибка: {mse:.6f}")

Результаты:
[5. 2. 1. 0.] -> -0.1186 (Истинное: 0.0)
[6. 3. 2. 1.] -> 0.1776 (Истинное: 0.0)
[4. 1. 1. 0.] -> 0.0515 (Истинное: 0.0)
[7. 4. 6. 3.] -> 1.1001 (Истинное: 1.0)
[8. 5. 5. 4.] -> 0.9489 (Истинное: 1.0)
[6. 3. 4. 2.] -> 0.8407 (Истинное: 1.0)
[3. 8. 7. 9.] -> 1.0272 (Истинное: 1.0)
[4. 9. 8. 8.] -> -0.0161 (Истинное: 0.0)
[2. 7. 6. 7.] -> 0.9881 (Истинное: 1.0)

Среднеквадратичная ошибка: 0.009716


In [2]:
import numpy as np

# Исходные данные
X = np.array([
    [5, 2, 1, 0], [6, 3, 2, 1], [4, 1, 1, 0],
    [7, 4, 6, 3], [8, 5, 5, 4], [6, 3, 4, 2],
    [3, 8, 7, 9], [4, 9, 8, 8], [2, 7, 6, 7],
], dtype=np.float32) / 10.0

y = np.array([0, 0, 0, 1, 1, 1, 1, 0, 1], dtype=np.float32)

# Нормализация (Z-score)
X_normalized = (X - X.mean(axis=0)) / (X.std(axis=0) + 1e-8)

# Функция для создания матрицы дизайна
def create_design_matrix(X):
    X1, X2, X3, X4 = X[:, 0], X[:, 1], X[:, 2], X[:, 3]
    return np.column_stack([
        np.ones(len(X), dtype=np.float32),
        X1, X2, X3, X4,
        X1**2, X2**2, X3**2, X4**2,
        X1*X2, X1*X3, X1*X4,
        X2*X3, X2*X4, X3*X4
    ])

design_matrix = create_design_matrix(X_normalized)

# Инициализация весов (Xavier/Glorot)
limit = np.sqrt(6 / (design_matrix.shape[1] + 1))
weights = np.random.uniform(-limit, limit, size=design_matrix.shape[1]).astype(np.float32)

# Параметры обучения
learning_rate = 0.1
lambda_reg = 0.001  # L2-регуляризация
n_iterations = 10000
best_mse = float('inf')
patience = 100
no_improvement = 0

# Обучение с ранней остановкой
for epoch in range(n_iterations):
    predictions = np.dot(design_matrix, weights)
    errors = predictions - y
    gradient = (np.dot(design_matrix.T, errors) + lambda_reg * weights) / len(y)
    weights -= learning_rate * gradient

    mse = np.mean(errors**2)
    if mse < best_mse:
        best_mse = mse
        no_improvement = 0
    else:
        no_improvement += 1
        if no_improvement >= patience:
            print(f"Ранняя остановка на итерации {epoch}")
            break

# Вывод результатов с округлением
print("Результаты:")
for i in range(len(X)):
    original_features = X[i] * 10
    prediction = np.dot(create_design_matrix(X_normalized[i:i+1]), weights)[0]
    print(f"{original_features} -> {prediction:.4f} (Истинное: {int(y[i])})")

print(f"\nСреднеквадратичная ошибка: {best_mse:.6f}")

Результаты:
[5. 2. 1. 0.] -> -0.0364 (Истинное: 0)
[6. 3. 2. 1.] -> 0.0378 (Истинное: 0)
[4. 1. 1. 0.] -> 0.0169 (Истинное: 0)
[7. 4. 6. 3.] -> 1.0109 (Истинное: 1)
[8. 5. 5. 4.] -> 0.9929 (Истинное: 1)
[6. 3. 4. 2.] -> 0.9771 (Истинное: 1)
[3. 8. 7. 9.] -> 1.0036 (Истинное: 1)
[4. 9. 8. 8.] -> -0.0015 (Истинное: 0)
[2. 7. 6. 7.] -> 0.9972 (Истинное: 1)

Среднеквадратичная ошибка: 0.000417


In [3]:
import numpy as np

# Исходные данные
X = np.array([
    [5, 2, 1, 0], [6, 3, 2, 1], [4, 1, 1, 0],
    [7, 4, 6, 3], [8, 5, 5, 4], [6, 3, 4, 2],
    [3, 8, 7, 9], [4, 9, 8, 8], [2, 7, 6, 7],
], dtype=np.float32) / 10.0

y = np.array([0, 0, 0, 1, 1, 1, 1, 0, 1], dtype=np.float32)

# Функция для создания матрицы дизайна (всех термов)
def create_design_matrix(X):
    # Создаем пустую матрицу 9x15
    design_matrix = np.zeros((9, 15), dtype=np.float32)

    # Заполняем каждый элемент вручную
    for i in range(9):
        x1, x2, x3, x4 = X[i][0], X[i][1], X[i][2], X[i][3]

        # bias term
        design_matrix[i][0] = 1.0

        # линейные термы
        design_matrix[i][1] = x1
        design_matrix[i][2] = x2
        design_matrix[i][3] = x3
        design_matrix[i][4] = x4

        # квадратичные термы
        design_matrix[i][5] = x1*x1
        design_matrix[i][6] = x2*x2
        design_matrix[i][7] = x3*x3
        design_matrix[i][8] = x4*x4

        # взаимодействия
        design_matrix[i][9] = x1*x2
        design_matrix[i][10] = x1*x3
        design_matrix[i][11] = x1*x4
        design_matrix[i][12] = x2*x3
        design_matrix[i][13] = x2*x4
        design_matrix[i][14] = x3*x4

    return design_matrix

# Создаем матрицу дизайна
design_matrix = create_design_matrix(X)

# Красиво выводим матрицу дизайна
print("\nМатрица дизайна (9x15):")
print("="*120)
print("Sample | Bias  | X1    X2    X3    X4    | X1²    X2²    X3²    X4²   | X1X2   X1X3   X1X4   X2X3   X2X4   X3X4")
print("-"*120)
for i in range(9):
    row = design_matrix[i]
    print(f"{i+1:2}     | {row[0]:.1f}  | "
          f"{row[1]:.2f}   {row[2]:.2f}   {row[3]:.2f}   {row[4]:.2f}   | "
          f"{row[5]:.3f}  {row[6]:.3f}  {row[7]:.3f}  {row[8]:.3f}  | "
          f"{row[9]:.3f}  {row[10]:.3f}  {row[11]:.3f}  {row[12]:.3f}  {row[13]:.3f}  {row[14]:.3f}")
print("="*120)

# Инициализация весов
weights = np.random.uniform(-0.1, 0.1, size=15).astype(np.float32)

# Параметры обучения
learning_rate = 0.1
n_iterations = 10000

# Обучение
for _ in range(n_iterations):
    predictions = np.dot(design_matrix, weights)
    errors = predictions - y
    gradient = np.dot(design_matrix.T, errors) / len(y)
    weights -= learning_rate * gradient

print("\nРезультаты:")
for i in range(9):
    print(f"{X[i]*10} -> {np.dot(design_matrix[i], weights):.4f} (Истинное: {y[i]})")

mse = np.mean((np.dot(design_matrix, weights) - y)**2)
print(f"\nСреднеквадратичная ошибка: {mse:.6f}")


Матрица дизайна (9x15):
Sample | Bias  | X1    X2    X3    X4    | X1²    X2²    X3²    X4²   | X1X2   X1X3   X1X4   X2X3   X2X4   X3X4
------------------------------------------------------------------------------------------------------------------------
 1     | 1.0  | 0.50   0.20   0.10   0.00   | 0.250  0.040  0.010  0.000  | 0.100  0.050  0.000  0.020  0.000  0.000
 2     | 1.0  | 0.60   0.30   0.20   0.10   | 0.360  0.090  0.040  0.010  | 0.180  0.120  0.060  0.060  0.030  0.020
 3     | 1.0  | 0.40   0.10   0.10   0.00   | 0.160  0.010  0.010  0.000  | 0.040  0.040  0.000  0.010  0.000  0.000
 4     | 1.0  | 0.70   0.40   0.60   0.30   | 0.490  0.160  0.360  0.090  | 0.280  0.420  0.210  0.240  0.120  0.180
 5     | 1.0  | 0.80   0.50   0.50   0.40   | 0.640  0.250  0.250  0.160  | 0.400  0.400  0.320  0.250  0.200  0.200
 6     | 1.0  | 0.60   0.30   0.40   0.20   | 0.360  0.090  0.160  0.040  | 0.180  0.240  0.120  0.120  0.060  0.080
 7     | 1.0  | 0.30   0.80   0.70   0.9